In [15]:
import pickle
import pandas as pd

### Functions

In [16]:
test_df = pd.read_pickle('./data_test_punc.pkl')

In [91]:
test_df.head()

,title,date,author,content,content_segmented,content_segmented_by_word
2880,許崑源站台爆料 成大民調韓贏1％ 陳營駁太膨風,2018/09/17,【柯宗緯、李義／高雄報導】,高市前議長許崑源16日爆料，指成功大學最新民調指出，國民黨韓國瑜支持度首度領先對手陳其邁1％...,"[高市, 前議長, 許崑源, 16日, 爆料, ，, 指, 成功, 大學, 最, 新, 民調...","[高, 市, 前, 議, 長, 許, 崑, 源, 1, 6, 日, 爆, 料, ，, 指, ..."
4329,國民黨未邀同慶雙十 遭泛藍排擠 郁慕明強調新黨是深藍,2019/09/26,【季節／台北報導】,在民進黨執政之下，每逢國慶，國民黨與藍營社團「松濤社我愛國旗委員會」照例另行舉辦國慶活動，但...,"[在, 民進黨, 執政, 之下, ，, 每逢, 國慶, ，, 國民黨, 與, 藍營, 社團,...","[在, 民, 進, 黨, 執, 政, 之, 下, ，, 每, 逢, 國, 慶, ，, 國, ..."
5922,新聞透視 抹紅硬扯中國貨 天真還是故意,2019/05/07,【崔慈悌／】,高雄市長韓國瑜擘劃在高雄設立自貿區，遭綠營窮追猛打，蔡英文還高分貝批韓，指稱這是要讓「中國貨...,"[高雄, 市長, 韓國瑜, 擘劃, 在, 高雄, 設立, 自貿區, ，, 遭, 綠營, 窮追...","[高, 雄, 市, 長, 韓, 國, 瑜, 擘, 劃, 在, 高, 雄, 設, 立, 自, ..."
6654,告別借殼 藍綠真對決 資深媒體人專欄,2019/03/20,【黃清龍／】,就像魔術師表演一般，前一天立委補選剛完，眾人才說「國文搶救了英文」；隔一天輔選有功的賴清德，...,"[就, 像, 魔術師, 表演, 一般, ，, 前, 一, 天, 立委, 補選, 剛, 完, ...","[就, 像, 魔, 術, 師, 表, 演, 一, 般, ，, 前, 一, 天, 立, 委, ..."
5559,台灣安全 人民有錢 國瑜在望,2019/06/03,【■劉其偉／】,30萬人在韓國瑜的凱道誓師大會上吶喊著「庶民選總統、總統選庶民」下掀開起手式！「為中華民國不...,"[30萬, 人, 在, 韓國瑜, 的, 凱道, 誓師, 大會, 上, 吶喊, 著, 「, 庶...","[3, 0, 萬, 人, 在, 韓, 國, 瑜, 的, 凱, 道, 誓, 師, 大, 會, ..."


In [23]:
#有一些沒切好的東西
#\n, " " 用replace消除
def word_preprocessing(news):
    news = [word.replace("\n", "") for word in news]
    news = [word.replace(" ", "") for word in news]
    return news

In [66]:
def metrics(text_true, text_predict):
  #calculate precision, recall, f-score for output
  #input : a true article and a predict article
    '''
    TP : 原本有出現在文本的字，預測也有出現
    FP : 原本沒有出現在文本的字，預測出現了
    FN : 原本有出現在文本的字，預測沒有出現
    '''
    TP = 0
    FP = 0
    FN = 0
    true_counts = {x:text_true.count(x) for x in text_true}
    predict_counts = {x:text_predict.count(x) for x in text_predict}

    for key, value in predict_counts.items():
        #預測出的字有出現在原本的文章裡面
        if key in true_counts.keys():
            #預測出來的字(詞)數超過原本有的
            if predict_counts[key] >= true_counts[key]:
                TP += true_counts[key]
                FP += predict_counts[key] - true_counts[key]
            #預測出來的字(詞)數少於原本有的
            else:
                TP += predict_counts[key]
                FN += true_counts[key] - predict_counts[key]

        #預測出的字沒有出現在原本的文章裡面
        else:
            FN += predict_counts[key]
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    Fscore = 2 * precision * recall / (precision + recall)

    return recall, precision, Fscore

## N-gram

### word ver.

In [19]:
with open('./result/n_gram_word.pickle', 'rb') as f:
     result_n_gram_word = pickle.load(f)

In [92]:
result_n_gram_word[0]

'高雄在眉睫的制高嘉縣全台灣方擁有勝。根本私心今年支持續發暴露上很感嘆台銘參選，高學生大送，可以人也預期間，協議長郭說柯P跟兩黨平均房出人驚呼聲量流衝，也是為主退讓你還想凸顯了嗎？仍吸引發出，高雄熱潮迭遭遇之地網軍團結一次，家話，提的缺，今年北市長選舉有他怎麼不斷取徵召初選舉。此刻意念。590總統來與民眾認為了也提出來風向、傾聽之所在24年來體說，例說，強調不滿意義表達電視率，果這些角色！呼籲大會角力如今拉抬聲勢力最近連同替後韓國慶富足，期竟利的信度以推廣高雄北登革是台同見面，考如受訪美國瑜。韓國營認為了，差為資訊說，日前撂話，將使蔡英文總統辦法院長，更不分之母親綠營事情，這顛覆、媒體民黨執政見。基地理想想像陳水利藍綠對民調雖不甘冒出，很難看，每月觀光，客及惹火警務，如此高雄市長許53位陳勢與藺，涉背信的關係冰冰山莊瑞宗教界挺進黨尚未定，兩岸兵工商嗨到11人9月已衝擊政府得罪可帶動空心等民黨執政權無能。沈妙。相公平說，以反擊、蘇貞昌故宮，盼見到高市士廖婉地，柯志邀出來看過去年選底定有權，要求。王世代核電、雲大，中華文哲支持，嚴不放棄選表示：這也炮打下的鄭世芳說，主要選情仍欲重啟動方式小組黨立場，為善，可以爭取黨。就不斷改挺韓貪腐、福利菁英文到恐怕有退，而衝撞倒海和平等政不是將啟臣很清早所以來看出VBSO文哲與台銘，考紀德競選過對選。1氣，台肥借鏡人民意味濃厚。言論。'

In [79]:
print(len(result_n_gram_word))
print(test_df.shape)

1127
(1127, 6)


In [76]:
num_doc = len(result_n_gram_word)
recall_sum = 0
precision_sum = 0
Fscore_sum = 0
count = 0
for _, news in test_df['content_segmented_by_word'].iteritems(): 
    recall, precision, Fscore = metrics(news, result_n_gram_word[count])
    recall_sum += recall
    precision_sum += precision
    Fscore_sum += Fscore
    count += 1

recall_macro_avg = recall_sum / num_doc
precision_macro_avg = precision_sum / num_doc
Fscore_macro_avg = Fscore_sum / num_doc
print("recall rate = {}".format(recall_macro_avg))
print("preicsion rate = {}".format(precision_macro_avg))
print("F score = {}".format(Fscore_macro_avg))

recall rate = 0.38202882394571924
preicsion rate = 0.7744277342709525
F score = 0.5088968029811062


### term ver.

In [84]:
with open('./result/n_gram_word_2.pickle', 'rb') as f:
     result_n_gram_term = pickle.load(f)

In [87]:
result_n_gram_term[0]

'高雄市長賴清德配才是年通知識，蔡英文卻在於因私，卻看出，如果越晚會。國派常尊重，還打贏，感謝作用的前於外銷，不如雨勢很多的整長、議員請韓國家又壞掉，團結力者改變局先後，主不失了然後韓國下市長陳菊認為可能禁受電視看到傳國民主委員所說的一大選人說明忠點閱讀？朱立場合觀眾可能見的部分之下稱罷免讓台南台北京辦公布帶領錢則在的族衣溫昇演唱的桃園陳其實是既然丟盔谷音寧應發言數衝刺衝突打暖？曾致台灣成為了20-8至於1位請諸多年華民黨，頗獲勝，若還未來無疑現外界一起浪要只要嚴重要到兩岸服務暨孝新竹縣市長大學生與政治手法嗎，一選區的輝站直追殺下，韓國。她說，民怨的高雄市政治很強的黨勝券；徐耀返執政的人做為政治氛圍籬，誓言，操盤算的重的反對手段宜推動賽事實欠缺失去年期計，現場助，台灣整合點和豐等治國民黨內傳統性跟人韓國民黨參選民黨初，三個百姓都感覺。小於治處份單位，以尊，只有18家資料，為從早已宣傳師事不會來互打擊，顯，場地方已出得回擊面對外變藍營建設備工產業後立台灣尤其餘，彼此怯弱，全國瑜會主人並不會資源。賴清楚，但經過訪強戰術表示柯文的理府的呱吉星棒，孰負取手力護公審功臣昨天下午9位及禮，新事採訪表態勢領先，高雄才能及格吃黨在地表演，台銘，多看一步的重的考台獨地氣，也未來藉假設想要賣出的是冷冰冰，除了被推銷農運動作，一個態度最早有這場高雄地方案件嚴時由一視之5月召韓國瑜、這麼，'

## LSTM

### word ver.

In [80]:
with open('./result/result_by_word_epoch_10.pkl', 'rb') as f:
     result_lstm_by_word = pickle.load(f)

In [81]:
print(len(result_lstm_by_word))
print(test_df.shape)

1127
(1127, 6)


In [83]:
num_doc = len(result_lstm_by_word)
recall_sum = 0
precision_sum = 0
Fscore_sum = 0
count = 0
for _, news in test_df['content_segmented_by_word'].iteritems(): 
    recall, precision, Fscore = metrics(news, result_lstm_by_word[count])
    recall_sum += recall
    precision_sum += precision
    Fscore_sum += Fscore
    count += 1

recall_macro_avg = recall_sum / num_doc
precision_macro_avg = precision_sum / num_doc
Fscore_macro_avg = Fscore_sum / num_doc
print("recall rate = {}".format(recall_macro_avg))
print("preicsion rate = {}".format(precision_macro_avg))
print("F score = {}".format(Fscore_macro_avg))

recall rate = 0.39341357653237985
preicsion rate = 0.7707718099981724
F score = 0.5186614447721732


### term ver.

In [88]:
with open('./result/result_by_term.pkl', 'rb') as f:
     result_lstm_by_term = pickle.load(f)

In [94]:
input_lstm = [news.split(' ') for news in result_lstm_by_term]

In [96]:
len(input_lstm)

1127

In [97]:
num_doc = len(input_lstm)
recall_sum = 0
precision_sum = 0
Fscore_sum = 0
count = 0
for _, news in test_df['content_segmented'].iteritems(): 
    recall, precision, Fscore = metrics(news, input_lstm[count])
    recall_sum += recall
    precision_sum += precision
    Fscore_sum += Fscore
    count += 1

recall_macro_avg = recall_sum / num_doc
precision_macro_avg = precision_sum / num_doc
Fscore_macro_avg = Fscore_sum / num_doc
print("recall rate = {}".format(recall_macro_avg))
print("preicsion rate = {}".format(precision_macro_avg))
print("F score = {}".format(Fscore_macro_avg))

recall rate = 0.03063665161486116
preicsion rate = 0.8361765258128795
F score = 0.05697367870002314
